In [10]:
import numpy as np
import pandas as pd
from scipy import misc

import torchvision.transforms as transforms
import os
import shutil
from tqdm import tqdm 

import os
from PIL import Image, ImageEnhance

In [11]:
# the folder for images
dir_train = 'E:/Jupyter NB/10 class Classification/Images'

# the directory of separeated images
image_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test'

In [12]:
t = pd.read_csv('E:/Jupyter NB/10 class Classification/train_metadata.csv')
v = pd.read_csv('E:/Jupyter NB/10 class Classification/val_metadata.csv')
#test = pd.read_csv('E:/Jupyter NB/10 class Classification/test_data.csv')

In [13]:
os.mkdir(image_dir + '/train')
for i in range(1, 20 + 1):
    os.mkdir(image_dir + '/train/' + str(i))

In [14]:
os.mkdir(image_dir + '/val')
for i in range(1, 20 + 1):
    os.mkdir(image_dir + '/val/' + str(i))

In [15]:
'''
os.mkdir(image_dir + '/test')
for i in range(1, 10 + 1):
    os.mkdir(image_dir + '/test/' + str(i))
'''

"\nos.mkdir(image_dir + '/test')\nfor i in range(1, 10 + 1):\n    os.mkdir(image_dir + '/test/' + str(i))\n"

In [16]:
val_transform = transforms.Compose([
    transforms.Scale(299, Image.LANCZOS),
    transforms.CenterCrop(299)
])

In [17]:
val_size = len(v)
val_size

400

In [18]:
# resize RGB images
for i, row in tqdm(v.loc[v.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # transform it
    image = val_transform(image)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

398it [00:03, 115.22it/s]


In [19]:
# resize grayscale images
for i, row in tqdm(v.loc[v.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # transform it
    image = val_transform(image)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    save_path = os.path.join(image_dir, 'val', str(row.category_number), row.img_name)
    image.save(save_path, 'jpeg')

2it [00:00, 143.22it/s]


In [20]:
enhancers = {
    0: lambda image, f: ImageEnhance.Color(image).enhance(f),
    1: lambda image, f: ImageEnhance.Contrast(image).enhance(f),
    2: lambda image, f: ImageEnhance.Brightness(image).enhance(f),
    3: lambda image, f: ImageEnhance.Sharpness(image).enhance(f)
}

factors = {
    0: lambda: np.random.uniform(0.4, 1.6),
    1: lambda: np.random.uniform(0.8, 1.2),
    2: lambda: np.random.uniform(0.8, 1.2),
    3: lambda: np.random.uniform(0.4, 1.6)
}

# randomly enhance images in random order
def enhance(image):
    order = [0, 1, 2, 3]
    np.random.shuffle(order)
    for i in order:
        f = factors[i]()
        image = enhancers[i](image, f)
    return image

In [21]:
train_transform_rare = transforms.Compose([
    transforms.Scale(384, Image.LANCZOS),
    transforms.RandomCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.Lambda(enhance)
])

train_transform = transforms.Compose([
    transforms.Scale(384, Image.LANCZOS),
    transforms.RandomCrop(299),
    transforms.RandomHorizontalFlip(),
])

In [22]:
# number of images in each category
class_counts = dict(t.category_name.value_counts())
np.save('class_counts.npy', class_counts)

In [23]:
# sample with replacement 100 images from each category
t = t.groupby('category_name', group_keys=False).apply(lambda x: x.sample(n=100, replace=True))
t.reset_index(drop=True, inplace=True)

In [24]:
train_size = len(t)
train_size

2000

In [25]:
#resize RGB images
for i, row in tqdm(t.loc[t.channels == 3].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # transform it
    if class_counts[row.category_name] < 100:
        image = train_transform_rare(image)
    else:
        image = train_transform(image)
    
    # save
    new_image_name = str(i) + '_' + row.img_name
    save_path = os.path.join(image_dir, 'train', str(row.category_number), new_image_name)
    image.save(save_path, 'jpeg')

1987it [00:23, 86.06it/s] 


In [26]:
# resize grayscale images
for i, row in tqdm(t.loc[t.channels == 1].iterrows()):
    # get image
    file_path = os.path.join(dir_train, row.directory, row.img_name)
    image = Image.open(file_path)
    
    # transform it
    if class_counts[row.category_name] < 100:
        image = train_transform_rare(image)
    else:
        image = train_transform(image)
    
    # convert to RGB
    array = np.asarray(image, dtype='uint8')
    array = np.stack([array, array, array], axis=2)
    image = Image.fromarray(array)
    
    # save
    new_image_name = str(i) + '_' + row.img_name
    save_path = os.path.join(image_dir, 'train', str(row.category_number), new_image_name)
    image.save(save_path, 'jpeg')

13it [00:00, 16.51it/s]
